In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/home/ec2-user/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').T
print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'automobile': 1, 'ship': 8, 'truck': 9, 'deer': 4, 'airplane': 0, 'horse': 7, 'cat': 3, 'frog': 6, 'bird': 2, 'dog': 5}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
print(X_validation.shape)
print(X_test.shape)

(5000, 3072)
(10000, 3072)


In [6]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[1]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
        
        return X_batch, y_batch

In [7]:
logdir = 'tf_logs/summary1'
iteration = 10000
batch_size = 128
learning_rate = 0.001

In [8]:
# try use a function to create a layer
# def create_conv(filters, input_layer):
#     conv = tf.layers.conv2d(inputs=input_layer, filters=filters, kernel_size=3, strides=1, 
#                                     padding = 'SAME', activation = tf.nn.relu, name='conv')
#     pool = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool')
#     return pool

In [9]:
with tf.device('/gpu:0'):
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, shape = (None, 3072))
        y = tf.placeholder(tf.int32, shape = (None))
        dropout_rate = tf.placeholder(tf.float32, shape=())
        training = tf.placeholder(tf.bool)
        input_layer = tf.reshape(X, [-1, 32, 32, 3])

    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv1')
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool1')

    with tf.name_scope('conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv2')
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2, padding='SAME', name='pool2')

    with tf.name_scope('fc') as scope:
        pool2_flat = tf.reshape(pool2, [-1, 8*8*64], name='pool2_flat')
        dense = tf.layers.dense(pool2_flat, units = 8*64, activation = tf.nn.relu, name='dense')
        dropout = tf.layers.dropout(dense, rate=dropout_rate, training=training, name='dropout')

    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(dropout, units = 10, name='logits')

    with tf.name_scope('loss') as scope:
        softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
        loss = tf.reduce_mean(softmax) 

    with tf.name_scope('train') as scope:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('eval') as scope:
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    with tf.name_scope('summary') as scope:
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logdir + '/test', tf.get_default_graph())

    with tf.name_scope('init') as scope:
        init = tf.global_variables_initializer()
    

In [10]:
def train(X_train, y_train, X_validation, y_validation, iteration):
    config = tf.ConfigProto()
    config.allow_soft_placement = True
    
    with tf.Session(config=config) as sess:
        sess.run(init)
        for step in range(iteration):
            X_batch, y_batch = get_batch(X_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                
            if step % 100 == 0:
                summary, loss_, acc = sess.run([merged, loss, accuracy],
                                               feed_dict={X: X_validation, y:y_validation, dropout_rate:0.3, training:False})
                test_writer.add_summary(summary, step)
                print('###################################')
                print('validation! after '+ str(step) + ' iterations' + 
                              'the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
                        
                summary, loss_, acc = sess.run([merged, loss, accuracy], 
                                               feed_dict={X: X_batch, y:y_batch,dropout_rate:0.3, training:False})
                train_writer.add_summary(summary, step)
                        
                print('training! after '+ str(step) + ' iterations' + 
                          'the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
            
            

In [11]:
train(X_train, y_train, X_validation, y_validation, iteration)

###################################
validation! after 0 iterationsthe loss is 2.31242, the accuracy is 0.107
training! after 0 iterationsthe loss is 2.30797, the accuracy is 0.09375
###################################
validation! after 100 iterationsthe loss is 2.30225, the accuracy is 0.1234
training! after 100 iterationsthe loss is 2.29957, the accuracy is 0.132812
###################################
validation! after 200 iterationsthe loss is 2.29592, the accuracy is 0.134
training! after 200 iterationsthe loss is 2.29026, the accuracy is 0.148438
###################################
validation! after 300 iterationsthe loss is 2.29113, the accuracy is 0.141
training! after 300 iterationsthe loss is 2.2878, the accuracy is 0.140625
###################################
validation! after 400 iterationsthe loss is 2.2871, the accuracy is 0.1474
training! after 400 iterationsthe loss is 2.28373, the accuracy is 0.195312
###################################
validation! after 500 iterationsth

###################################
validation! after 4400 iterationsthe loss is 1.92438, the accuracy is 0.3262
training! after 4400 iterationsthe loss is 1.90598, the accuracy is 0.289062
###################################
validation! after 4500 iterationsthe loss is 1.91838, the accuracy is 0.3208
training! after 4500 iterationsthe loss is 1.87991, the accuracy is 0.328125
###################################
validation! after 4600 iterationsthe loss is 1.91239, the accuracy is 0.3264
training! after 4600 iterationsthe loss is 1.83461, the accuracy is 0.34375
###################################
validation! after 4700 iterationsthe loss is 1.91223, the accuracy is 0.3298
training! after 4700 iterationsthe loss is 1.9204, the accuracy is 0.351562
###################################
validation! after 4800 iterationsthe loss is 1.90279, the accuracy is 0.3386
training! after 4800 iterationsthe loss is 1.94325, the accuracy is 0.367188
###################################
validation! afte

###################################
validation! after 8800 iterationsthe loss is 1.74266, the accuracy is 0.3938
training! after 8800 iterationsthe loss is 1.70295, the accuracy is 0.359375
###################################
validation! after 8900 iterationsthe loss is 1.74171, the accuracy is 0.3868
training! after 8900 iterationsthe loss is 1.55556, the accuracy is 0.515625
###################################
validation! after 9000 iterationsthe loss is 1.73728, the accuracy is 0.392
training! after 9000 iterationsthe loss is 1.50735, the accuracy is 0.453125
###################################
validation! after 9100 iterationsthe loss is 1.73179, the accuracy is 0.3968
training! after 9100 iterationsthe loss is 1.67931, the accuracy is 0.453125
###################################
validation! after 9200 iterationsthe loss is 1.7326, the accuracy is 0.3952
training! after 9200 iterationsthe loss is 1.56832, the accuracy is 0.507812
###################################
validation! afte